In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import datetime as dt

purchases = pd.read_csv('../DATA/purchases.csv.gzip', compression = 'gzip')

products = pd.read_csv('../DATA/products.csv')

purchases = purchases.merge(products, how = 'left', on = 'product_id')

%%time
pivot_table = pd.pivot_table(purchases, 
                             values='product_quantity', 
                             index=['num_transaction_id', 
                                    'transaction_datetime',
                                    'num_client_id'], 
                             columns = ['level_2'],
                             aggfunc = np.sum).fillna(0).reset_index()

pivot_table['transaction_datetime'] = pivot_table['transaction_datetime'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
pivot_table = pivot_table.set_index('transaction_datetime')

pivot_table.to_pickle('pivot.pickle')

%%time
target = pivot_table[['num_client_id', 'fb3d79909eb455c6cd6183b956598a12']]

target['target'] = target.groupby('num_client_id') \
                         .shift(-3) \
                         .rolling(3) \
                         .max() 

target['target'] = target['target'].astype(bool).astype(int)
target = target.reset_index().drop(['fb3d79909eb455c6cd6183b956598a12'], axis = 1)

target.to_pickle('../DATA/target_next.pickle')